In [1]:
import pandas as pd
import numpy as np

In [2]:
def get_df(fname):
    df = pd.read_csv('data/'+fname, index_col=0)    
    df = df.T
    return df

fname = 'Test_relia100_Tyler_20191107_122335.csv'
df = get_df(fname)
df.head()

case name,system reliability,CO2 price ($/kgCO2),norm. demand to 1,demand file,mean demand (kW),fixed cost natgas ($/kW/h),var cost natgas ($/kW/h),fixed cost natgas_ccs ($/kW/h),var cost natgas_ccs ($/kW/h),fixed cost solar ($/kW/h),...,dispatch wind (kW),curtailment wind (kW),capacity nuclear (kW),dispatch nuclear (kW),curtailment nuclear (kW),capacity storage (kW),energy storage (kW),dispatch to storage (kW),dispatch from storage (kW),dispatch unmet demand (kW)
CONUS_BAs_MEM1,1.0,0.0,True,demand_series_Tyler_mean_CONUS_from_BAs_for_ME...,1.0,0.0118,0.039,0.0273,0.0566,0.0195,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CONUS_BAs_MEM2,1.0,0.0,True,demand_series_Tyler_mean_CONUS_from_BAs_for_ME...,1.0,-1.0,-1.0,-1.0,-1.0,0.0195,...,0.4459940212939382,0.004490914262882603,0.9208561643882295,0.3799285816886027,0.5409275826996269,1.1816560192138257,0.24556450618286993,0.0067262952304316,0.006053385763851404,0.0
CONUS_BAs_MEM3,1.0,0.0,True,demand_series_Tyler_mean_CONUS_from_BAs_for_ME...,1.0,-1.0,-1.0,-1.0,-1.0,0.0195,...,0.5756139830276367,1.5417316987706284,0.0,0.0,0.0,4.868800872585583,0.7853434487838803,0.004576981825761849,0.004118388351654084,0.0
CONUS_BAs_impute1,1.0,0.0,True,demand_series_Tyler_mean_CONUS_from_BAs_mean_i...,0.9999999999999999,0.0118,0.039,0.0273,0.0566,0.0195,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CONUS_BAs_impute2,1.0,0.0,True,demand_series_Tyler_mean_CONUS_from_BAs_mean_i...,0.9999999999999999,-1.0,-1.0,-1.0,-1.0,0.0195,...,0.14946975155557116,0.06494599688520508,0.2983969972619471,0.020691120774641907,0.27770587648730516,32.363171048755966,6.895125350569129,0.37066258192446794,0.33358846328912184,0.0


In [3]:
def demand_info(df, idx):
    dem = pd.read_csv('~/Downloads/Tyler_Data_3/'+df.loc[idx, 'demand file'], header=5)
    max_ = np.max(dem['demand'])
    min_ = np.min(dem['demand'])
    mean_ = np.mean(dem['demand'])
    std_ = np.std(dem['demand'])
    #idxs = dem['demand'] == max_
    #print(dem.loc[idxs, 'year'],dem.loc[idxs, 'month'],dem.loc[idxs, 'day'],dem.loc[idxs, 'hour'],dem.loc[idxs, 'demand'])
    return dem, max_, min_, mean_, std_


def print_case(fname, tgt_case, scenario, print_cap=True):
    df = get_df(fname)
    if scenario == '1':
        test = 'EIA Base'
    elif scenario == '2':
        test = 'No NatGas'
    elif scenario == '3':
        test = 'No NatGas, no nuclear'
    
    order = ['Scenario 1','Scenario 2','Scenario 3']
    save_map = {}
    save_map_dem = {}
    
    for idx in df.index:
        #if idx[-4:] == 'MEM1':
        if tgt_case in idx and idx[-1] == scenario:
            if 'MEM' in idx:
                case_code = 'Scenario 3'
            elif 'impute' in idx:
                case_code = 'N/A'
                continue
            elif 'REGIONS' in idx:
                case_code = 'Scenario 2'
            else:
                case_code = 'Scenario 1'
            
            dem, max_, min_, mean_, std_ = demand_info(df, idx)
            
            val = 0
            ng = round(float(df.loc[idx, 'capacity natgas (kW)'])*mean_/1000)
            wind = round(float(df.loc[idx, 'capacity wind (kW)'])*mean_/1000)
            solar = round(float(df.loc[idx, 'capacity solar (kW)'])*mean_/1000)
            nuke = round(float(df.loc[idx, 'capacity nuclear (kW)'])*mean_/1000)
            storage = round(float(df.loc[idx, 'capacity storage (kW)'])*mean_/1000)
            save_map[case_code] = f'{tgt_case}:{test}:{case_code}:{ng}:{wind}:{solar}:{nuke}:{storage}'
            save_map_dem[case_code] = f'{tgt_case}:{case_code}:{mean_}:{max_}:{round(max_/mean_*100)}:{min_}:{round(min_/mean_*100)}'#:{round(std_/mean_*100,2)}'
    
    if print_cap:
        for s in order:
            print(save_map[s])
    else:
        for s in order:
            print(save_map_dem[s])
    return


fname = 'Test_relia100_Tyler_20191107_122335.csv'
print_case(fname, 'CONUS', '1')

CONUS:EIA Base:Scenario 1:3769:0:0:0:0
CONUS:EIA Base:Scenario 2:618:0:377:0:0
CONUS:EIA Base:Scenario 3:717:0:0:0:0


In [4]:
print("Region:Test:Case:NatGas:Wind:Solar:Nuclear:Storage")
tests = ['CONUS','EASTERN','WESTERN','TEXAS']
for test in tests:
    for i in ['1','2','3']:
        print_case(fname, test, i)

Region:Test:Case:NatGas:Wind:Solar:Nuclear:Storage
CONUS:EIA Base:Scenario 1:3769:0:0:0:0
CONUS:EIA Base:Scenario 2:618:0:377:0:0
CONUS:EIA Base:Scenario 3:717:0:0:0:0
CONUS:No NatGas:Scenario 1:0:203:1951:133:14677
CONUS:No NatGas:Scenario 2:0:532:388:388:574
CONUS:No NatGas:Scenario 3:0:483:290:422:542
CONUS:No NatGas, no nuclear:Scenario 1:0:604:2030:0:14638
CONUS:No NatGas, no nuclear:Scenario 2:0:2204:658:0:2434
CONUS:No NatGas, no nuclear:Scenario 3:0:2269:757:0:2233
EASTERN:EIA Base:Scenario 1:533:0:0:0:0
EASTERN:EIA Base:Scenario 2:534:0:0:0:0
EASTERN:EIA Base:Scenario 3:533:0:0:0:0
EASTERN:No NatGas:Scenario 1:0:76:113:399:461
EASTERN:No NatGas:Scenario 2:0:77:112:400:462
EASTERN:No NatGas:Scenario 3:0:77:106:403:445
EASTERN:No NatGas, no nuclear:Scenario 1:0:1807:512:0:3489
EASTERN:No NatGas, no nuclear:Scenario 2:0:1766:514:0:3755
EASTERN:No NatGas, no nuclear:Scenario 3:0:1818:504:0:3575
WESTERN:EIA Base:Scenario 1:3420:0:0:0:0
WESTERN:EIA Base:Scenario 2:139:0:0:0:0
WESTER

In [5]:
print("Region:Test:Mean Demand (MW):Maximum Demand (MW):Max/Mean (%):Minimum Demand (MW):Min/Mean (%)")#:Std/Mean (%)")
tests = ['CONUS','EASTERN','WESTERN','TEXAS']
for test in tests:
    for i in ['1',]:#'2','3']:
        print_case(fname, test, i, False)

Region:Test:Mean Demand (MW):Maximum Demand (MW):Max/Mean (%):Minimum Demand (MW):Min/Mean (%)
CONUS:Scenario 1:455351.9020301095:3768848:828:218042:48
CONUS:Scenario 2:450401.451908455:935346:208:202281:45
CONUS:Scenario 3:458583.6417274939:717466:156:311432:68
EASTERN:Scenario 1:331283.5068810827:533327:161:145984:44
EASTERN:Scenario 2:332490.289651764:533675:161:169064:51
EASTERN:Scenario 3:333669.9602341849:533327:160:218141:65
WESTERN:Scenario 1:83172.61960158151:3419601:4111:37044:45
WESTERN:Scenario 2:83212.72574513382:139310:167:46568:56
WESTERN:Scenario 3:83636.5920772506:139310:167:59270:71
TEXAS:Scenario 1:40895.77554744526:73264:179:0:0
TEXAS:Scenario 2:41209.52976733576:73264:178:0:0
TEXAS:Scenario 3:41277.09447801106:73264.0:177:25146.0:61


In [6]:
### This cell shows why the max demand does not align for Eastern between S1,2,3.
### It is b/c for S1 and S3, I aggregate from the BA level and exclude SEC and OVEC
### while EIA method is based on regions and includes SEC and OVEC

def return_all_regions():
    return ['AEC', 'AECI', 'CPLE', 'CPLW',
    'DUK', 'FMPP', 'FPC',
    'FPL', 'GVL', 'HST', 'ISNE',
    'JEA', 'LGEE', 'MISO', 'NSB',
    'NYIS', 'OVEC', 'PJM', 'SC',
    'SCEG', 'SEC', 'SOCO',
    'SPA', 'SWPP', 'TAL', 'TEC',
    'TVA', 'ERCO',
    'AVA', 'AZPS', 'BANC', 'BPAT',
    'CHPD', 'CISO', 'DOPD',
    'EPE', 'GCPD', 'IID',
    'IPCO', 'LDWP', 'NEVP', 'NWMT',
    'PACE', 'PACW', 'PGE', 'PNM',
    'PSCO', 'PSEI', 'SCL', 'SRP',
    'TEPC', 'TIDC', 'TPWR', 'WACM',
    'WALC', 'WAUW']

regions = return_all_regions()
regions = ['SEC','OVEC']

regions.sort()
print(len(regions))
for region in regions:
    print(region)

    file_path = '~/data4/{}.csv'.format(region)
    df = pd.read_csv(file_path,
                    dtype={'demand (MW)':np.float64},
                    parse_dates=True, na_values=['MISSING', 'EMPTY'])
    print(df.loc[df['time'] == '20160811T21Z'])

2
OVEC
              time  year  month  day  hour  demand (MW)  forecast demand (MW)
9784  20160811T21Z  2016      8   11    21        121.0                 109.0
SEC
              time  year  month  day  hour  demand (MW)  forecast demand (MW)
9784  20160811T21Z  2016      8   11    21        227.0                 278.0
